In [1]:
#Default
import pandas as pd
pd.options.display.float_format = '{:,}'.format
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
def reset(df):
    cols = df.columns
    return df.reset_index()[cols]
def print_counts(df):
    cols = df.columns
    for each in cols:
        print(each)
        print(df[each].value_counts())
        print('______________________________________')

# ~
#Default Ending

print('OK !')

OK !


In [2]:
import requests
import pandas as pd
import time # จำเป็นต้องใช้เพื่อหน่วงเวลา

# CoinGecko API URL
URL = 'https://api.coingecko.com/api/v3/coins/markets'

headers = {
    'Accepts': 'application/json',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def get_all_crypto_gecko():
    all_crypto_list = []
    page = 1
    
    print("กำลังเริ่มดึงข้อมูล... กรุณารอสักครู่ (อาจใช้เวลาหลายนาที)")

    while True:
        parameters = {
            'vs_currency': 'thb',
            'order': 'market_cap_desc',
            'per_page': 250,        # สูงสุด 250 ต่อหน้า
            'page': page,           # เลขหน้าปัจจุบัน
            'sparkline': 'false',
            'price_change_percentage': '1h,24h,7d'
        }

        try:
            response = requests.get(URL, headers=headers, params=parameters)
            
            # กรณีโดนจำกัดการเข้าถึง (Rate Limit)
            if response.status_code == 429:
                print(f"⚠️ หน้า {page}: โดนจำกัดความเร็ว (Rate Limit) รอ 60 วินาที...")
                break
                time.sleep(60) # พักยาวหน่อยแล้วลองใหม่
                continue
            
            response.raise_for_status()
            data = response.json()

            # ถ้าไม่มีข้อมูลกลับมา แสดงว่าดึงครบทุกหน้าแล้ว -> จบการทำงาน
            if not data:
                print("✅ ดึงข้อมูลครบทุกหน้าแล้ว")
                break

            print(f"กำลังดึงข้อมูลหน้า {page} (จำนวน {len(data)} เหรียญ)...")

            for coin in data:
                all_crypto_list.append({
                    'ชื่อเหรียญ': coin['name'],
                    'สัญลักษณ์': coin['symbol'].upper(),
                    'ราคา (THB)': coin['current_price'],
                    '1 ชม. %': coin.get('price_change_percentage_1h_in_currency'),
                    '24 ชม. %': coin.get('price_change_percentage_24h_in_currency'),
                    '7 วัน %': coin.get('price_change_percentage_7d_in_currency'),
                    'มูลค่าตลาด': coin['market_cap']
                })
            
            page += 1
            
            # *** สำคัญมาก: ต้องหน่วงเวลาเพื่อไม่ให้โดนแบน IP ***
            # สำหรับ Public API แนะนำ 1.5 - 3 วินาทีต่อหน้า
            time.sleep(2) 

        except requests.exceptions.RequestException as e:
            print(f"❌ Error ในหน้า {page}: {e}")
            break
            
    # สร้าง DataFrame เมื่อ Loop จบ
    if all_crypto_list:
        df = pd.DataFrame(all_crypto_list)
        pd.options.display.float_format = '{:,.2f}'.format
        return df
    else:
        return None

# if __name__ == "__main__":
#     df = get_all_crypto_gecko()
    
#     if df is not None:
#         print("\nสรุปข้อมูล:")
#         print(f"จำนวนเหรียญทั้งหมดที่ดึงได้: {len(df)} เหรียญ")
        
        # แสดงตัวอย่าง 10 เหรียญแรก
#         print(df.head(10).to_string(index=False))
        
        # บันทึกเป็นไฟล์ CSV หรือ Excel เพื่อเอาไปใช้งานต่อ (เพราะข้อมูลมันเยอะมากดูใน Terminal ไม่ไหว)
        # df.to_csv('all_crypto_thb.csv', index=False, encoding='utf-8-sig')
        # print("บันทึกไฟล์ all_crypto_thb.csv เรียบร้อย")

In [3]:
import requests

def get_binance_futures_symbols():
    # URL สำหรับ Binance USDS-M Futures API
    url = "https://fapi.binance.com/fapi/v1/exchangeInfo"
    
    try:
        # ส่ง request ไปยัง API
        response = requests.get(url)
        response.raise_for_status()  # ตรวจสอบว่า request สำเร็จหรือไม่
        
        data = response.json()
        
        # ดึงเฉพาะ Symbol ที่มีสถานะการเทรดเป็น 'TRADING'
        symbols = [
            symbol['symbol'] 
            for symbol in data['symbols'] 
            if symbol['status'] == 'TRADING'
        ]
        
        return symbols

    except Exception as e:
        print(f"เกิดข้อผิดพลาด: {e}")
        return []

# เรียกใช้งานฟังก์ชัน
futures_symbols = get_binance_futures_symbols()

# print(f"พบทั้งหมด {len(futures_symbols)} symbols")
# print("ตัวอย่าง 10 รายชื่อแรก:")
# print(futures_symbols[:10])

In [5]:
df = get_all_crypto_gecko()

df['symbol'] = df['สัญลักษณ์'] + 'USDT'

df = df[df['symbol'].isin(futures_symbols)]

df = df.sort_values(by = 'มูลค่าตลาด', ascending = False)

df = reset(df)

df = df.groupby('symbol').first().reset_index()

df = df.sort_values(by = 'มูลค่าตลาด', ascending = False)

df = reset(df)

df['abs_7d'] = abs(df['7 วัน %'])

df = df.sort_values(by = 'abs_7d', ascending = False)

df = reset(df)

cols = ['ชื่อเหรียญ', 'ราคา (THB)', '1 ชม. %','24 ชม. %', '7 วัน %', 'มูลค่าตลาด']
df.head(20)[cols]

กำลังเริ่มดึงข้อมูล... กรุณารอสักครู่ (อาจใช้เวลาหลายนาที)
กำลังดึงข้อมูลหน้า 1 (จำนวน 250 เหรียญ)...
กำลังดึงข้อมูลหน้า 2 (จำนวน 250 เหรียญ)...
กำลังดึงข้อมูลหน้า 3 (จำนวน 250 เหรียญ)...
กำลังดึงข้อมูลหน้า 4 (จำนวน 250 เหรียญ)...
⚠️ หน้า 5: โดนจำกัดความเร็ว (Rate Limit) รอ 60 วินาที...


,ชื่อเหรียญ,ราคา (THB),1 ชม. %,24 ชม. %,7 วัน %,มูลค่าตลาด
0,River,377.29,-6.25,24.52,217.39,7415955719
1,Yei Finance,12.14,-1.38,-7.82,61.91,1581160476
2,Apro,5.43,-1.85,2.41,60.60,1253249645
3,Tradoor,56.97,1.32,15.13,53.87,817523059
4,Canton,4.88,-2.41,-1.00,53.05,179240493583
5,Flow,2.61,-0.13,-7.54,-51.93,4257349620
6,SQD,2.91,0.36,3.57,51.22,2736321593
7,STBL,2.03,-1.49,14.17,50.43,1013681236
8,Everlyn,5.11,-0.62,-2.53,50.39,1068653972
9,Audiera,31.90,-1.75,-26.74,-48.67,4474674532


In [6]:
'''
Please transcribe the data from the attached image into a Markdown table in Thai. Follow these specific formatting rules strictly:

1. **Rank (First Column):** Rename the header to "ลำดับ". The numbers in the image start at 0, but I want you to start counting from 1 (increment each rank by 1).
2. **Coin Name:** Transliterate the English name into Thai phonetics, followed by the original English name in parentheses.
   - Example: "River" -> "ริเวอร์ (River)"
   - Example: "Collect on Fanable" -> "คอลเลกต์ ออน แฟนเอเบิล (Collect on Fanable)"
3. **Price:** Keep the values in USD ($).
4. **Percentages (1h, 24h, 7d):**
   - Add a Green Circle emoji (🟢) before positive numbers.
   - Add a Red Circle emoji (🔴) before negative numbers.
   - Include the "%" sign.
5. **Market Cap:** simplify the large numbers to make them readable in Thai.
   - Use "ล้าน" for Million.
   - Use "พันล้าน" for Billion.
   - Example: "208,835,611.93" should become "208.84 ล้าน".
6. **General:** Ensure the table is clean and aligned.
Output the final result as a table.

'''

'\nPlease transcribe the data from the attached image into a Markdown table in Thai. Follow these specific formatting rules strictly:\n\n1. **Rank (First Column):** Rename the header to "ลำดับ". The numbers in the image start at 0, but I want you to start counting from 1 (increment each rank by 1).\n2. **Coin Name:** Transliterate the English name into Thai phonetics, followed by the original English name in parentheses.\n   - Example: "River" -> "ริเวอร์ (River)"\n   - Example: "Collect on Fanable" -> "คอลเลกต์ ออน แฟนเอเบิล (Collect on Fanable)"\n3. **Price:** Keep the values in USD ($).\n4. **Percentages (1h, 24h, 7d):**\n   - Add a Green Circle emoji (🟢) before positive numbers.\n   - Add a Red Circle emoji (🔴) before negative numbers.\n   - Include the "%" sign.\n5. **Market Cap:** simplify the large numbers to make them readable in Thai.\n   - Use "ล้าน" for Million.\n   - Use "พันล้าน" for Billion.\n   - Example: "208,835,611.93" should become "208.84 ล้าน".\n6. **General:** Ens